<a href="https://colab.research.google.com/github/vberezina/machine-learning-basics/blob/main/practices/LR4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ЛАБОРАТОРНАЯ РАБОТА №4. Наивный байесовский классификатор (Naive Bayes)

## Теоретический минимум



### **Почему он «Наивный»?**
Алгоритм делает допущение, что **все признаки независимы друг от друга** относительно целевой переменной. В реальности это почти никогда не так (например, в тексте слова «машинное» и «обучение» часто идут рядом).

### **Теорема Байеса**

В основе алгоритма лежит классическая теорема вероятности. Для задачи классификации она записывается так:

$$P(y \mid X) = \frac{P(X \mid y) \cdot P(y)}{P(X)}$$

Где:
* $P(y \mid X)$ — **Апостериорная вероятность:** вероятность того, что объект относится к классу $y$ при данных признаках $X$
* $P(y)$ — **Априорная вероятность:** насколько часто этот класс встречается в выборке.
* $P(X \mid y)$ — **Правдоподобие:** вероятность встретить такие признаки у представителя класса $y$

* $P(X)$ — **Свидетельство:** вероятность встретить такие признаки в принципе (при сравнении классов этот множитель обычно отбрасывают, так как он одинаков для всех).



### **Гауссовское распределение**

**Gaussian Naive Bayes** предполагает, что значения признаков распределены по нормальному (Гауссовскому) закону. Чтобы описать это распределение для каждого класса, нам нужно знать всего два параметра:

1. **Среднее ($\mu$):** центр «колокола».

2. **Дисперсия ($\sigma^2$):** ширина «колокола».


Для классификации мы ищем класс $y$, который максимизирует вероятность:
$$P(y \mid x_1, \dots, x_n) \propto P(y) \prod_{i=1}^{n} P(x_i \mid y)$$

В гауссовском варианте вероятность признака $P(x_i \mid y)$ рассчитывается по формуле нормального распределения:

$$P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma_y^2}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma_y^2}\right)$$

### **Проблема малых чисел и Log-sum-exp**

Когда признаков много, перемножение большого количества вероятностей (чисел типа $0.01$) приводит к экстремально маленьким значениям, которые компьютер округляет до нуля.
Чтобы этого избежать, мы переходим к **логарифмам**. Вместо умножения вероятностей мы складываем их логарифмы:
$$\ln(P) = \ln(P(y)) + \sum \ln(P(x_i \mid y))$$


## Учебная задача


In [1]:
import numpy as np

class NaiveBayes:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        # Инициализируем среднее, дисперсию и априорные вероятности
        self.mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self.var = np.zeros((n_classes, n_features), dtype=np.float64)
        self.priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0)
            self.priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self.classes):
            # Используем логарифмы, чтобы избежать зануления при умножении малых чисел
            prior = np.log(self.priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = posterior + prior
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        """Probability Density Function (Гауссовское распределение)"""
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        # Добавляем небольшое число (1e-9) в знаменатель, чтобы избежать деления на ноль
        numerator = np.exp(- (x - mean)**2 / (2 * var + 1e-9))
        denominator = np.sqrt(2 * np.pi * var + 1e-9)
        return numerator / denominator

# --- Пример использования ---
if __name__ == "__main__":
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import make_classification

    # Генерируем данные
    X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=123)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

    nb = NaiveBayes()
    nb.fit(X_train, y_train)
    predictions = nb.predict(X_test)

    accuracy = np.sum(predictions == y_test) / len(y_test)
    print(f"Точность модели: {accuracy * 100:.2f}%")



Точность модели: 96.50%


## Задания


### Задание №1. Классификатор

Реализуйте алгоритм Gaussian Naive Bayes, используя только библиотеку numpy. Ваш класс должен следовать стандартному интерфейсу `fit/predict`:

1. В методе `fit` рассчитайте среднее значение ($\mu$), дисперсию ($\sigma^2$) и априорную вероятность ($P(y)$) для каждого класса.

2. В методе `predict` реализуйте вычисление функции плотности вероятности (PDF) для нормального распределения.

3. Используйте логарифмирование вероятностей, чтобы избежать проблемы арифметического недополнения (чисел, близких к нулю).






### Задание №2. Обучение

Проверьте работоспособность собственного классификатора на искусственно сгенерированных данных. Для этого:

1. Сгенерируйте выборку из 1000 объектов и 10 признаков для бинарной классификации.

2. Разделите данные на обучающую и тестовую выборки в соотношении 80/20.

3. Обучите модель на тренировочных данных и выведите итоговую точность (accuracy) на тестовых данных.



## Контрольные вопросы

1. Почему классификатор называется «Наивным»? К каким последствиям для точности это может привести?

2. Что такое априорная вероятность ($P(y)$) в контексте кода? Как она вычисляется в методе fit?

3. Какое распределение (кроме Гауссовского) можно использовать для классификации текстовых документов?

4. Как изменится работа алгоритма, если один из признаков будет сильно коррелировать с другим? Нарушит ли это работу кода технически?


